## Training with NumPy
It's also possible to integrate with NumPy *during training.* If desired, the entire estimator could be in NumPy.

This is adapted from [PyTorch documentation on SciPy extensions][1].

This demo might be desired because there are two methods for "convolutions":

* direct (faster for typical PyTorch use case)
* Fourier (faster for cases with two large images)

SciPy's convolve makes a smart choice about which convolution method to use (it relies on SciPy's [choose_conv_method][2]).


[1]:https://pytorch.org/tutorials/advanced/numpy_extensions_tutorial.html
[2]:https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.choose_conv_method.html#scipy.signal.choose_conv_method

In [2]:
import random
from scipy.signal import convolve, correlate
import torch
import numpy as np

In [4]:
from torch.autograd import Function
from torch.nn.modules.module import Module
from torch.nn.parameter import Parameter

class ScipyConvFunction(Function):
    @staticmethod
    def forward(ctx, input, filter, bias):
        # detach so we can cast to NumPy
        input, filter, bias = input.detach(), filter.detach(), bias.detach()
        result = correlate(input.numpy(), filter.numpy(), mode='valid')
        result += bias.numpy()
        ctx.save_for_backward(input, filter, bias)
        return torch.as_tensor(result, dtype=input.dtype)

    @staticmethod
    def backward(ctx, grad_output):
        grad_output = grad_output.detach()
        input, filter, bias = ctx.saved_tensors
        grad_output = grad_output.numpy()
        grad_bias = np.sum(grad_output, keepdims=True)
        grad_input = convolve(grad_output, filter.numpy(), mode='full')
        
        grad_filter = correlate(input.numpy(), grad_output, mode='valid')
        return torch.from_numpy(grad_input), torch.from_numpy(grad_filter).to(torch.float), torch.from_numpy(grad_bias).to(torch.float)


class ScipyConv(Module):
    def __init__(self, filter_width, filter_height):
        super(ScipyConv, self).__init__()
        self.filter = Parameter(torch.randn(filter_width, filter_height))
        self.bias = Parameter(torch.randn(1, 1))

    def forward(self, input):
        return ScipyConvFunction.apply(input, self.filter, self.bias)


In [6]:
module = ScipyConv(3, 3)
in_ = torch.randn(10, 10, requires_grad=True)
out = module(in_)
out.backward(torch.randn(8, 8))

In [7]:
in_.grad[:3, :3]

tensor([[-0.0893, -0.3339, -0.1441],
        [ 0.0684,  0.0345, -0.9547],
        [ 0.6083, -0.1076, -1.8432]])

Great, gradients can be obtained. But are they correct?

Let's check.

In [8]:
from torch.autograd.gradcheck import gradcheck

In [11]:
moduleConv = ScipyConv(3, 3)

in_ = [torch.randn(20, 20, dtype=torch.double, requires_grad=True)]
test = gradcheck(moduleConv, in_)
print("Are the gradients correct?", test)

Are the gradients correct? True
